In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re


# Question 1

In [6]:

cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities=cities.apply(lambda x: x.apply(lambda y: re.findall("[\w]*(?=\[.*\])|[\w ]*",y)[0]))
cities["NHL"] = cities["NHL"].replace({"RangersIslandersDevils": "Rangers,Islanders,Devils",
                                           "KingsDucks": "Kings,Ducks",
                                           "Red Wings": "Red,Wings", 
                                           "Maple Leafs": "Maple,Leafs", 
                                           "Blue Jackets": "Blue,Jackets",
                                           "Golden Knights": "Golden,Knights" })
cities["NHL"]=cities["NHL"].apply(lambda x: x.split(","))
cities = cities.explode("NHL")
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,Rangers
0,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,Islanders
0,New York City,20153634,GiantsJets,YankeesMets,KnicksNets,Devils
1,Los Angeles,13310447,RamsChargers,DodgersAngels,LakersClippers,Kings
1,Los Angeles,13310447,RamsChargers,DodgersAngels,LakersClippers,Ducks


In [20]:
# cleaning the nhl_df dataframe
nhl_df=pd.read_csv("nhl.csv")
nhl_df=nhl_df[nhl_df["year"]==2018]
nhl_df["team"]=nhl_df["team"].apply(lambda x: x.replace("*","").split(" ")[-1])
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
0,Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,Atlantic Division,2018,NHL
1,Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL


In [23]:
nhl_df.drop(index=nhl_df[nhl_df["team"]=="Division"].index, inplace=True)
nhl_df.head()

,team,GP,W,L,OL,PTS,PTS%,GF,GA,SRS,SOS,RPt%,ROW,year,League
1,Lightning,82,54,23,5,113,.689,296,236,0.66,-0.07,.634,48,2018,NHL
2,Bruins,82,50,20,12,112,.683,270,214,0.62,-0.07,.610,47,2018,NHL
3,Leafs,82,49,26,7,105,.640,277,232,0.49,-0.06,.567,42,2018,NHL
4,Panthers,82,44,30,8,96,.585,248,246,-0.01,-0.04,.537,41,2018,NHL
5,Wings,82,30,39,13,73,.445,217,255,-0.48,-0.01,.341,25,2018,NHL


In [76]:
# merge the dataframes
df = pd.merge(cities, nhl_df, left_on="NHL", right_on="team")
df = df[["Metropolitan area", "Population (2016 est.)[8]", "NHL", "team", "W", "L"]]
df["WL"]=df["W"].astype("int")/(df["W"].astype("int") + df["L"].astype("int"))
df.rename(columns={"Population (2016 est.)[8]":"Pop"}, inplace=True)
df["Pop"] = df["Pop"].astype("float")
df.head(10)

,Metropolitan area,Pop,NHL,team,W,L,WL
0,New York City,20153634,Rangers,Rangers,34,39,0.465753
1,New York City,20153634,Islanders,Islanders,35,37,0.486111
2,New York City,20153634,Devils,Devils,44,29,0.602740
3,Los Angeles,13310447,Kings,Kings,45,29,0.608108
4,Los Angeles,13310447,Ducks,Ducks,44,25,0.637681
5,San Francisco Bay Area,6657982,Sharks,Sharks,45,27,0.625000
6,Chicago,9512999,Blackhawks,Blackhawks,33,39,0.458333
7,Dallas,7233323,Stars,Stars,42,32,0.567568
8,Washington,6131977,Capitals,Capitals,49,26,0.653333
9,Philadelphia,6070500,Flyers,Flyers,42,26,0.617647


In [182]:
new_df=df.groupby(["Metropolitan area","Pop"])["WL"].agg(WL=np.mean).reset_index()
new_df.set_index("Metropolitan area",inplace=True)
new_df.head()

,Pop,WL
Metropolitan area,,
Atlanta,5789700.0,0.4375
Baltimore,2798886.0,0.6250
Boston,4794447.0,0.6875
Buffalo,1132804.0,0.3750
Charlotte,2474314.0,0.4375


# Question 2

In [132]:
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities=cities.apply(lambda x: x.apply(lambda y: re.findall("[\w]*(?=\[.*\])|[\w ]*",y)[0]))
cities["NBA"] = cities["NBA"].replace({"KnicksNets": "Knicks,Nets",
                                           "LakersClippers": "Lakers,Clippers",
                                           "Trail Blazers": "Trail,Blazers"})
cities["NBA"] = cities["NBA"].apply(lambda x: x.split(","))
cities = cities.explode("NBA")
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets,YankeesMets,Knicks,RangersIslandersDevils
0,New York City,20153634,GiantsJets,YankeesMets,Nets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers,DodgersAngels,Lakers,KingsDucks
1,Los Angeles,13310447,RamsChargers,DodgersAngels,Clippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders,GiantsAthletics,Warriors,Sharks


In [133]:
# cleaning the nba_df dataframe
nba_df=pd.read_csv("nba.csv")
nba_df=nba_df[nba_df["year"]==2018]
nba_df["team"]=nba_df["team"].apply(lambda x: re.findall("[\w \w]*",x)[0].split(" ")[-1])
nba_df.head()

,team,W,L,W/L%,GB,PS/G,PA/G,SRS,year,League
0,Raptors,59,23,0.72,—,111.7,103.9,7.29,2018,NBA
1,Celtics,55,27,0.6709999999999999,4.0,104.0,100.4,3.23,2018,NBA
2,76ers,52,30,0.634,7.0,109.8,105.3,4.3,2018,NBA
3,Cavaliers,50,32,0.61,9.0,110.9,109.9,0.59,2018,NBA
4,Pacers,48,34,0.585,11.0,105.6,104.2,1.18,2018,NBA


In [146]:
# merging dataframes
df = pd.merge(cities, nba_df, left_on="NBA", right_on="team")
df = df[["Metropolitan area", "Population (2016 est.)[8]", "NBA", "team", "W", "L"]]
df["WL"]=df["W"].astype("int")/(df["W"].astype("int") + df["L"].astype("int"))
df.rename(columns={"Population (2016 est.)[8]":"Pop"}, inplace=True)
df=df.groupby(["Metropolitan area","Pop"])["WL"].agg(WL=np.mean).reset_index()
df["Pop"] = df["Pop"].astype("float")
df.set_index("Metropolitan area",inplace=True)

numpy.float64

# Question 3

In [208]:
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities=cities.apply(lambda x: x.apply(lambda y: re.findall("[\w]*(?=\[.*\])|[\w ]*",y)[0]))
cities["MLB"] = cities["MLB"].replace({"Blue Jays": "Blue,Jays", 
                                           "CubsWhite Sox": "Cubs,White Sox", 
                                           "DodgersAngels": "Dodgers,Angels", 
                                           "GiantsAthletics": "Giants,Athletics", 
                                           "YankeesMets": "Yankees,Mets"})
cities["MLB"] = cities["MLB"].apply(lambda x: x.split(","))
cities = cities.explode("MLB")
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,GiantsJets,Yankees,KnicksNets,RangersIslandersDevils
0,New York City,20153634,GiantsJets,Mets,KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,RamsChargers,Dodgers,LakersClippers,KingsDucks
1,Los Angeles,13310447,RamsChargers,Angels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ersRaiders,Giants,Warriors,Sharks


In [206]:
# cleaning the mlb_df dataframe
mlb_df=pd.read_csv("mlb.csv")
mlb_df=mlb_df[mlb_df["year"]==2018]
mlb_df["team"]=mlb_df["team"].apply(lambda x: x.split(" ")[-2]+" "+x.split(" ")[-1] if "Sox" in x else x.split(" ")[-1])
mlb_df

,team,W,L,W-L%,GB,year,League
0,Red Sox,108,54,0.667,--,2018,MLB
1,Yankees,100,62,0.617,8.0,2018,MLB
2,Rays,90,72,0.556,18.0,2018,MLB
3,Jays,73,89,0.451,35.0,2018,MLB
4,Orioles,47,115,0.290,61.0,2018,MLB
5,Indians,91,71,0.562,--,2018,MLB
6,Twins,78,84,0.481,13.0,2018,MLB
7,Tigers,64,98,0.395,27.0,2018,MLB
8,White Sox,62,100,0.383,29.0,2018,MLB
9,Royals,58,104,0.358,33.0,2018,MLB


In [207]:
# merging dataframes
df = pd.merge(cities, mlb_df, left_on="MLB", right_on="team")
df = df[["Metropolitan area", "Population (2016 est.)[8]", "MLB", "team", "W", "L"]]
df["WL"]=df["W"].astype("int")/(df["W"].astype("int") + df["L"].astype("int"))
df.rename(columns={"Population (2016 est.)[8]":"Pop"}, inplace=True)
df=df.groupby(["Metropolitan area","Pop"])["WL"].agg(WL=np.mean).reset_index()
df["Pop"] = df["Pop"].astype("float")
df.set_index("Metropolitan area",inplace=True)
df

,Pop,WL
Metropolitan area,,
Atlanta,5789700.0,0.555556
Baltimore,2798886.0,0.290123
Boston,4794447.0,0.666667
Chicago,9512999.0,0.482769
Cincinnati,2165139.0,0.413580
Cleveland,2055612.0,0.561728
Dallas,7233323.0,0.413580
Denver,2853077.0,0.558282
Detroit,4297617.0,0.395062


# Question 4

In [164]:
cities=pd.read_html("wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities=cities.apply(lambda x: x.apply(lambda y: re.findall("[\w]*(?=\[.*\])|[\w ]*",y)[0]))
cities["NFL"] = cities["NFL"].replace({"GiantsJets": "Giants,Jets",
                                           "RamsChargers": "Rams,Chargers",
                                           "49ersRaiders": "49ers,Raiders"
                                           })
cities["NFL"] = cities["NFL"].apply(lambda x: x.split(","))
cities = cities.explode("NFL")
cities.head()

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,Giants,YankeesMets,KnicksNets,RangersIslandersDevils
0,New York City,20153634,Jets,YankeesMets,KnicksNets,RangersIslandersDevils
1,Los Angeles,13310447,Rams,DodgersAngels,LakersClippers,KingsDucks
1,Los Angeles,13310447,Chargers,DodgersAngels,LakersClippers,KingsDucks
2,San Francisco Bay Area,6657982,49ers,GiantsAthletics,Warriors,Sharks


In [180]:
# cleaning the nfl_df dataframe
nfl_df=pd.read_csv("nfl.csv")
nfl_df=nfl_df[nfl_df["year"]==2018]
nfl_df["team"]=nfl_df["team"].apply(lambda x: re.findall("[\w ]*",x)[0].split(" ")[-1])
nfl_df.head()

,DSRS,L,League,MoV,OSRS,PA,PD,PF,SRS,SoS,T,W,W-L%,team,year
0,AFC East,AFC East,NFL,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,AFC East,East,2018
1,2.1,5,NFL,6.9,3.1,325,111,436,5.2,-1.8,0,11,.688,Patriots,2018
2,-5.2,9,NFL,-7.1,-3.6,433,-114,319,-8.8,-1.7,0,7,.438,Dolphins,2018
3,-0.6,10,NFL,-6.6,-6.3,374,-105,269,-6.9,-0.3,0,6,.375,Bills,2018
4,-5.9,12,NFL,-6.8,-2.0,441,-108,333,-7.8,-1.1,0,4,.250,Jets,2018


In [179]:
# merging dataframes
df = pd.merge(cities, nfl_df, left_on="NFL", right_on="team")
df = df[["Metropolitan area", "Population (2016 est.)[8]", "NFL", "team", "W", "L"]]
df["WL"]=df["W"].astype("int")/(df["W"].astype("int") + df["L"].astype("int"))
df.rename(columns={"Population (2016 est.)[8]":"Pop"}, inplace=True)
df=df.groupby(["Metropolitan area","Pop"])["WL"].agg(WL=np.mean).reset_index()
df["Pop"] = df["Pop"].astype("float")
df.set_index("Metropolitan area",inplace=True)
len(df)

29